# Deploying model into AWS cloud environment using Sagemaker.

Steps:

Model Deployment:
- [ ] Login to sagemaker endpoint. 
- [ ] Convert keras to proto buf format from h5
- [ ] Push updated model to S3 bucket
- [ ] Deploy an endpoint in sagemaker

Inference process:
- [ ] Establish a lambda function triggered by API gateway (Code provided)
- [ ] Notebook entry to generate s3 signed urls
- [ ] Upload file to s3 using signed urls
- [ ] Infer endpoint to infer on the uploaded file
- [ ] Visualize the image and results


In [ ]:
# Upgrade available version of sagemaker

!pip install -U sagemaker

In [ ]:
# import sagemaker and tensorflow model to load the model from s3 bucket.

import sagemaker
from sagemaker.tensorflow import TensorFlowModel


In [ ]:
# Tensorflow imports needed for model conversion.

import tensorflow as tf
import tensorflow.compat.v1.keras.backend as K

from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants


In [ ]:
# Account related constants

BUCKET = "smoke-dataset-bucket"
EXECUTION_ROLE = 'arn:aws:iam::574347909231:role/igarss-sagemaker-role'

In [ ]:
model_filename = <model filename> # smoke_wmts_ref_3layer.h5.tar.gz
framework_version = <tensorflow version> # 2.9.1


In [ ]:
model = TensorFlowModel(
    model_data=f"s3://{BUCKET}/{model_filename}",
    framework_version=framework_version,
    role=EXECUTION_ROLE
)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
MODEL_VERSION = '1'
EXPORT_DIRECTORY = 'export/Servo/{}'


def convert_to_proto_buf(model_name)
    loaded_model = load_model(model_name)
    model_version = '1'
    export_dir =  EXPORT_DIRECTORY.format(model_version)
    builder = builder.SavedModelBuilder(export_dir)
    tf.compat.v1.disable_eager_execution()
    signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})
    builder.add_meta_graph_and_variables(
    sess=K.get_session(), tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

In [ ]:
signature

In [ ]:
model_path = 'export/Servo/1/'
!saved_model_cli show --all --dir {model_path}

In [ ]:
import tarfile
model_archive = 'model.tar.gz'
with tarfile.open(model_archive, mode='w:gz') as archive:
    archive.add('export', recursive=True) 

from sagemaker import get_execution_role
from sagemaker import Session
role = get_execution_role()
sess = Session()
bucket = sess.default_bucket()
    
    
# upload model artifacts to S3
model_data = sess.upload_data(path=model_archive, key_prefix='model')

In [ ]:
import tensorflow as tf
print(tf.__version__)  # This notebook runs on TensorFlow 1.15.x or earlier
tf_framework_version = tf.__version__

In [ ]:
from sagemaker.tensorflow.serving import Model


In [ ]:
instance_type = 'ml.t2.medium'

In [ ]:
sm_model = Model(model_data=model_data, 
                 framework_version='2.8',
                 role=role)

In [ ]:
%%time
uncompiled_predictor = sm_model.deploy(
        initial_instance_count=1, 
        instance_type=instance_type, 
        endpoint_name="<yourname>-sagemaker-endpoint"
    )   

In [ ]:
uncompiled_predictor

In [ ]:
data = np.random.randn(1, 256, 256, 6)

In [ ]:
import numpy as np
data = np.random.randn(1, 256, 256, 6)

In [ ]:
deployed_model_preds = uncompiled_predictor.predict(data)

### Testing deployed model

In [ ]:
np.asarray(deployed_model_preds['predictions']).shape

In [ ]:
original_model_preds = loaded_model.predict(data)

In [ ]:
difference = deployed_model_preds['predictions'] - original_model_preds

In [ ]:
difference